In [ ]:
from datetime import datetime
import pandas as pd

def calculate_and_delete(exit_gate_df, entry_gate_df_path, license_plate):
  """Calculates the time difference between entry and exit for a license plate and deletes entries if under 5 minutes.

  Args:
      exit_gate_df: The DataFrame for storing exit gate data.
      entry_gate_df_path: The path to the entry gate data Excel file.
      license_plate: The license plate to process.
  """
  timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  new_entry = {'License Plate': license_plate, 'Timestamp': timestamp}
  exit_gate_df = exit_gate_df.append(new_entry, ignore_index=True)  # Add exit entry

  # Read entry gate data from Excel (handle first run)
  try:
    entry_gate_df = pd.read_excel(entry_gate_df_path)
  except FileNotFoundError:
    print("Entry gate data file not found. Time difference calculation skipped.")
    entry_gate_df = pd.DataFrame(columns=['License Plate', 'Timestamp'])  # Create empty DataFrame on first run

  if entry_gate_df.empty:
    print("No entries found in entry gate data. Time difference calculation skipped.")
    return

  # Find matching entry in entry gate data (modify search logic if needed)
  matching_entry = entry_gate_df[entry_gate_df['License Plate'] == license_plate]

  if matching_entry.empty:
    print(f"License plate {license_plate} not found in entry gate data.")
    return

  # Calculate time difference in minutes
  entry_timestamp = matching_entry['Timestamp'].iloc[0]
  time_delta = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S') - datetime.strptime(entry_timestamp, '%Y-%m-%d %H:%M:%S')
  time_difference_in_minutes = time_delta.total_seconds() / 60

  if time_difference_in_minutes < 5:
    # Delete entries from both DataFrames
    entry_gate_df = entry_gate_df.drop(matching_entry.index)
    exit_gate_df = exit_gate_df.drop(exit_gate_df.index[-1])  # Delete the latest exit entry
    print(f"License plate {license_plate} spent less than 5 minutes (approximately {time_difference_in_minutes:.2f} minutes). Entries deleted from both files.")
  else:
    print(f"License plate {license_plate} spent {time_difference_in_minutes:.2f} minutes. No deletion performed")


In [ ]:
def add_entry(exit_gate_df, license_plate):
  """Adds a new entry with the given license plate and current timestamp to the DataFrame.

  Args:
      entry_gate_df: The DataFrame for storing entry gate data.
      license_plate: The license plate for the new entry.
  """
  timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  new_entry = {'License Plate': license_plate, 'Timestamp': timestamp}
  exit_gate_df = exit_gate_df.append(new_entry, ignore_index=True)  # Add entry to DataFrame

  # Try to read existing data from Excel (avoid overwriting on first run)
  try:
    existing_data = pd.read_excel("entry_gate_data.xlsx")
    entry_gate_df = exit_gate_df.append(existing_data, ignore_index=True)  # Combine existing and new data
  except FileNotFoundError:
    pass  # Ignore error on first run, new file will be created

  # Save DataFrame to Excel
  entry_gate_df.to_excel("entry_gate_data.xlsx", index=False)


# Example usage
exit_gate_df = pd.DataFrame(columns=['License Plate', 'Timestamp'])  # Initialize empty DataFrame
add_entry(exit_gate_df, "ABC123")

# Simulate reading more license plates (replace with your actual reading logic)
# ...